In [1]:
### Import libraries
import openai
import os
import time
from datetime import datetime
import pandas as pd
import clingo
from clingo.control import Control
import json
from yaml import load, Loader
from helper import *
from stories import *
from pipelines import *
import psycopg2

In [2]:
with open("config.yml","r") as config:
    data = load(config, Loader=Loader)
    api_key = data["apikey"]
    openai.api_key  = api_key

In [3]:
now = datetime.now()
date_time = now.strftime("%Y%m%d")

## Load Projects and Calculate Semantic Similarity

In [4]:
projects = get_useful_projects()
for p in projects:
    print(p['story'])

creek
minotaur
seeknumbers
sudoku
yosenabe
hop
lights


In [5]:
file = "SimilarityDataframe.csv"
if os.path.exists(file):
    df = pd.read_csv(file)
    print("Loaded Similarity Dataframe")
    if len(projects) != df.shape[0]:
        print("Calculating Similarity Dataframe")
        df = get_similarity_dataframe(projects)
        df.to_csv("SimilarityDataframe.csv", index=False)
        print("Loaded Similarity Dataframe")
else:
    print("Calculating Similarity Dataframe")
    df = get_similarity_dataframe(projects)
    df.to_csv("SimilarityDataframe.csv", index=False)

Loaded Similarity Dataframe


In [6]:
df.loc[:, df.columns != 'embeddings']

,stories,problems,cos_creek,cos_minotaur,cos_seeknumbers,cos_sudoku,cos_yosenabe,cos_hop,cos_lights
0,creek,The goal of the game is to blacken some cells ...,1.000,0.805,0.916,0.828,0.855,0.872,0.892
1,minotaur,There is a player in a labyrinth. The goal is ...,0.805,1.000,0.813,0.752,0.795,0.837,0.807
2,seeknumbers,"Given a square grid, the goal is to construct ...",0.916,0.813,1.000,0.838,0.875,0.913,0.901
3,sudoku,The task of this project is to solve a Sudoku ...,0.828,0.752,0.838,1.000,0.902,0.843,0.855
4,yosenabe,The task of this project is to solve the Japan...,0.855,0.795,0.875,0.902,1.000,0.872,0.870
5,hop,The goal of the puzzle is to find a path betwe...,0.872,0.837,0.913,0.843,0.872,1.000,0.894
6,lights,The puzzle involves illuminating a two-dimensi...,0.892,0.807,0.901,0.855,0.870,0.894,1.000


## Connect to the Database

In [7]:
conn_string = "dbname=thesis user=postgres password=postgres"
conn = psycopg2.connect(conn_string)
print("Connection established")

Connection established


In [8]:
def save_in_database(conn, prompt1, prompt2, encoding, story, story_1, story_2, version, file):
    cursor = conn.cursor()
    command = "INSERT INTO experiments (prompt1, prompt2, encoding, story, story_1, story_2, version, file) VALUES ('%(prompt1)s', '%(prompt2)s', '%(encoding)s', '%(story)s', '%(story_1)s', '%(story_2)s', '%(version)s', '%(file)s');" % {"prompt1": prompt1, "prompt2": prompt2, "encoding": encoding, "story": story, "story_1": story_1, "story_2":story_2, "version":version, "file":file}
    try:
        cursor.execute(command)
        conn.commit()
        cursor.close()
    except Exception as err:
        print(command)
        print(f"Unexpected {err=}, {type(err)=}")
        conn.commit()
        cursor.close()
        pass

## Run experiments by Version

In [9]:
def run_main_pipeline(projects,df,conn,version=1):
    for project in projects:
        query = dict(project)
        example_1, example_2 = get_best_context(df,projects,project["story"])
        story = query["story"]
        story_1 = example_1["story"]
        story_2 = example_2["story"]
        print("Working Pipeline for story " + story + ". Context: " + story_1 + " and " + story_2)
        answer, prompts = pipeline(example_1,example_2,query,version=version)
        name_of_file = story+"_from_"+story_1+"_and_"+story_2+"_v"+str(version) + "_" + date_time + ".lp"
        path_of_file = os.path.join("generated_solutions", name_of_file)
        save_in_database(conn,prompts[0].replace("'",""), prompts[1].replace("'",""), answer.replace("'",""), story, story_1, story_2, version, name_of_file)
        with open(path_of_file,"w") as file:
            file.write(answer)

In [12]:
version = 1
run_main_pipeline(projects,df,conn,version)

Working Pipeline for story creek. Context: seeknumbers and lights
Working Pipeline for story minotaur. Context: seeknumbers and hop
Working Pipeline for story seeknumbers. Context: creek and hop
Working Pipeline for story sudoku. Context: yosenabe and lights
Working Pipeline for story yosenabe. Context: seeknumbers and sudoku
Working Pipeline for story hop. Context: seeknumbers and lights
Working Pipeline for story lights. Context: seeknumbers and hop


In [10]:
version = 2
run_main_pipeline(projects,df,conn,version)

Working Pipeline for story creek. Context: seeknumbers and lights
Working Pipeline for story minotaur. Context: seeknumbers and hop
Working Pipeline for story seeknumbers. Context: creek and hop
Working Pipeline for story sudoku. Context: yosenabe and lights
Working Pipeline for story yosenabe. Context: seeknumbers and sudoku
Working Pipeline for story hop. Context: seeknumbers and lights
Working Pipeline for story lights. Context: seeknumbers and hop


In [13]:
version = 3
run_main_pipeline(projects,df,conn,version)

Working Pipeline for story creek. Context: seeknumbers and lights
Working Pipeline for story minotaur. Context: seeknumbers and hop
Working Pipeline for story seeknumbers. Context: creek and hop
Working Pipeline for story sudoku. Context: yosenabe and lights
Working Pipeline for story yosenabe. Context: seeknumbers and sudoku
Working Pipeline for story hop. Context: seeknumbers and lights
Working Pipeline for story lights. Context: seeknumbers and hop


In [10]:
version = 4
run_main_pipeline(projects,df,conn,version)

Working Pipeline for story creek. Context: seeknumbers and lights
Working Pipeline for story minotaur. Context: seeknumbers and hop
Working Pipeline for story seeknumbers. Context: creek and hop
Working Pipeline for story sudoku. Context: yosenabe and lights
Working Pipeline for story yosenabe. Context: seeknumbers and sudoku
Working Pipeline for story hop. Context: seeknumbers and lights
Working Pipeline for story lights. Context: seeknumbers and hop


In [10]:
version = 5
run_main_pipeline(projects,df,conn,version)

Working Pipeline for story creek. Context: seeknumbers and lights
Working Pipeline for story minotaur. Context: seeknumbers and hop
Working Pipeline for story seeknumbers. Context: creek and hop
Working Pipeline for story sudoku. Context: yosenabe and lights
Working Pipeline for story yosenabe. Context: seeknumbers and sudoku
Working Pipeline for story hop. Context: seeknumbers and lights
Working Pipeline for story lights. Context: seeknumbers and hop


In [11]:
version = 6
run_main_pipeline(projects,df,conn,version)

Working Pipeline for story creek. Context: seeknumbers and lights
Working Pipeline for story minotaur. Context: seeknumbers and hop
Working Pipeline for story seeknumbers. Context: creek and hop
Working Pipeline for story sudoku. Context: yosenabe and lights
Working Pipeline for story yosenabe. Context: seeknumbers and sudoku
Working Pipeline for story hop. Context: seeknumbers and lights
Working Pipeline for story lights. Context: seeknumbers and hop


In [ ]:
version = 7
run_main_pipeline(projects,df,conn,version)

In [9]:
version = 8
run_main_pipeline(projects,df,conn,version)

Working Pipeline for story creek. Context: sudoku and yosenabe
Working Pipeline for story minotaur. Context: yosenabe and hop
Working Pipeline for story seeknumbers. Context: creek and lights
Working Pipeline for story sudoku. Context: creek and yosenabe
Working Pipeline for story yosenabe. Context: creek and sudoku
Working Pipeline for story hop. Context: creek and yosenabe
Working Pipeline for story lights. Context: creek and seeknumbers


In [13]:
version = 9
run_main_pipeline(projects,df,conn,version)

Working Pipeline for story yosenabe. Context: seeknumbers and sudoku
